<a href="https://colab.research.google.com/github/alinealinealine/GPT-Pilot/blob/main/src/Finetuning_with_GPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tune GPT-3 for AIMM narrative ex-ante

OpenAI's GPT-3 is a natural language model trained on large set of training data. It can be used for various tasks, including to generate data. 

However, the model is generalist in nature and thus, not fit for specialised tasks in its original or vaniall version. However, will a bit of finetuning it can be used for more specialised tasks such as generating AIMM text. 

The fine-tuning happens via OpenAI's API to fine tune GPT-3. 

## Installing dependencies and libraries

In [ ]:
!pip install -Uq openai wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.7 MB/s eta 0:00:00


In [ ]:
import openai
import wandb
from pathlib import Path
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
#Entering API Credentials
openai.api_key_path = "./api.txt"

## Dataset Prepration

The dataset was processed in R by scrapping relevant documents and cleaning it into the JSON format required to finetune the mdoel. The dataset are split based on sector and portion of AIMM narrative it is expected to generate. 

1. Sector:
  1. FIG
  2. MAS
  3. CDF
  4. INR
2. Section of AIMM narrative
  1. Project narrative
  2. Market narrative 
  3. Indicators

In addition different variation of prompts are also explored - creating different models. 

## Model naming convention

In order to keep track of the models they are to be named using the following convention: "SSS-IN-GEN-XXXX"
* SSS: Refers to Sector of the model's focus
* IN: Refers to model input, can be BP for Board Papers and GE for Generic documents
* GEN: Refers to which section the model is trying to generate. Can be one of the following:
  * PRO: Project narrative
  * MAR: Market narrative
  * IND: Indicators
* XXXX: Refers to the number of the model - as various models might be created to accomodate various prompts. This can also be alpha numeric.




In [ ]:
project_name = 'FIG-BP-PRO'
model_name = 'FIG-BP-PRO-0001'

# Fine Tuning 

Fine tuning involves the following steps:
1. Preparing the dataset: Datset is here split into training and validation sets. Before the split - the prompts are also shared with OpenAI to see if they are aligned with the requirements for finetuning.
2. FineTuning: This is where the split datasets are shared with OpenAI for finetuning of the GPT model. The final model is saved and can be accessed both here and on OpenAI playground.  

## Preping the dataset

In [ ]:
# create a job for splitting dataset
run = wandb.init(project=project_name, job_type='split dataset')

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# download full dataset
dataset_path = "./"+model_name+".json"

In [ ]:
!head $dataset_path

{"prompt":"DEVELOPMENT IMPACT Summary The Project has an AIMM rating of [Satisfactory] based on an AIMM score of [35]. On an unadjusted basis (without a likelihood factor), the full potential AIMM score could reach [46]. The most significant expected project-level outcome is increased access to finance for VSE/SMEs and WSMEs. Beyond the Project, IFC anticipates that it has the potential to promote greater market competitiveness by demonstrating the commercial viability of lending to underserved VSE/SME segments.\n\n###\n\n","completion":" Assessment of Project Outcomes The project outcomes rating is [Strong] with a [Medium] likelihood of achievement. The likelihood assessment is based on the following: (i) the Bank management’s strategic commitment to growing its SME business contribution from 27% to 43% by 2021; (ii) integrated advisory services support that will help the Bank improve the use of the RSF, and thereby help expand its portfolio; (iii) the sound quality of the SME portfol

In [ ]:
!openai tools fine_tunes.prepare_data -f $dataset_path

Analyzing...

- Your JSON file appears to be in a JSONL format. Your file will be converted to JSONL format
- Your file contains 326 prompt-completion pairs
- There are 10 examples that are very long. These are rows: [25, 79, 127, 128, 133, 162, 193, 267, 272, 309]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- All prompts end with suffix `\n\n###\n\n`
- All completions end with suffix `\n[END]`

Based on the analysis we will perform the following actions:
- [Necessary] Your format `JSON` will be converted to `JSONL`
- [Recommended] Remove 10 long examples [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `./FIG-BP-PRO-0001_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "./FIG-BP-PRO-0001_prepared.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `\n\n###\n\n` for

In [ ]:
dataset_path = "./"+model_name+"_prepared.jsonl"
# check number of samples
!wc -l $dataset_path

316 ./FIG-BP-PRO-0001_prepared.jsonl


Splitting it into training and testing set randomly with 25% going to testing set. 
* Training Set = 75% 
* Validation Set = 25% 

Also, logging the files into W&B for recordkeeping. 

In [ ]:
df = pd.read_json(dataset_path, orient='records', lines=True)
df_train, df_test = train_test_split(df,test_size = 0.25,random_state = 42, shuffle = False)
df_train.to_json("./"+model_name+"_train.jsonl", orient='records', lines=True)
df_test.to_json("./"+model_name+"_test.jsonl", orient='records', lines=True)

#Logging the files and tables into W&B 
table_train = wandb.Table(dataframe=df_train)
table_valid = wandb.Table(dataframe=df_test)

# Create artifacts
artifact_train = wandb.Artifact(model_name+"_train.jsonl", type='training_files', metadata={'samples': df_train.shape[0]})
artifact_train.add_file(model_name+"_train.jsonl")
artifact_train.add(table_train, model_name+"_train.jsonl")

artifact_valid = wandb.Artifact(model_name+"_test.jsonl", type='validation_files', metadata={'samples': df_test.shape[0]})
artifact_valid.add_file(model_name+"_test.jsonl")
artifact_valid.add(table_valid, model_name+"_test.jsonl")

# Log files
run.log_artifact(artifact_train)
run.log_artifact(artifact_valid)

Closing our dataprep run

In [ ]:
# keep entity for reference of artifact later 
entity = wandb.run.entity
wandb.finish()

## Fine Tuning the model


In [ ]:
train_file = "./"+model_name+"_train.jsonl"
valid_file = "./"+model_name+"_test.jsonl"

Defining Hyper parameters: 

Using the default hyper parameters by OpenAI, replacing model with Divinci 003.

In [ ]:
#Defining hyper parameters (using the default ones)
model = 'davinci'  # using the best model : davinci
n_epochs = 4
batch_size = 4
learning_rate_multiplier = 0.1
prompt_loss_weight = 0.1

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "<API-Key>"

In [ ]:
!openai api fine_tunes.create \
    -t $train_file \
    -v $valid_file \
    -m $model \
    --n_epochs $n_epochs \
    --batch_size $batch_size \
    --learning_rate_multiplier $learning_rate_multiplier \
    --prompt_loss_weight $prompt_loss_weight \
    --suffix $model_name

Upload progress: 100% 1.28M/1.28M [00:00<00:00, 1.29Git/s]
Uploaded file from ./FIG-BP-PRO-0001_train.jsonl: file-0Y4ys5RxjsytU9TwoEt9TQ8d
Upload progress: 100% 480k/480k [00:00<00:00, 512Mit/s]
Uploaded file from ./FIG-BP-PRO-0001_test.jsonl: file-hCfsdVKbyXlRDoUbplGADQoV
Created fine-tune: ft-Jim3RgfBVLSs8YTGfCwHxkFU
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-07 19:14:35] Created fine-tune: ft-Jim3RgfBVLSs8YTGfCwHxkFU

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-Jim3RgfBVLSs8YTGfCwHxkFU



In [ ]:
!openai api fine_tunes.follow -i ft-Jim3RgfBVLSs8YTGfCwHxkFU

[2023-03-07 19:14:35] Created fine-tune: ft-Jim3RgfBVLSs8YTGfCwHxkFU
[2023-03-07 19:20:47] Fine-tune costs $30.63
[2023-03-07 19:20:48] Fine-tune enqueued. Queue number: 22
[2023-03-07 19:23:36] Fine-tune is in the queue. Queue number: 21
[2023-03-07 19:25:40] Fine-tune is in the queue. Queue number: 20
[2023-03-07 19:27:03] Fine-tune is in the queue. Queue number: 19
[2023-03-07 19:28:22] Fine-tune is in the queue. Queue number: 18
[2023-03-07 19:28:38] Fine-tune is in the queue. Queue number: 17
[2023-03-07 19:30:42] Fine-tune is in the queue. Queue number: 16
[2023-03-07 19:31:30] Fine-tune is in the queue. Queue number: 15
[2023-03-07 19:32:16] Fine-tune is in the queue. Queue number: 14
[2023-03-07 19:32:17] Fine-tune is in the queue. Queue number: 13
[2023-03-07 19:34:17] Fine-tune is in the queue. Queue number: 12
[2023-03-07 19:34:43] Fine-tune is in the queue. Queue number: 11
[2023-03-07 19:34:45] Fine-tune is in the queue. Queue number: 10
[2023-03-07 19:35:22] Fine-tune is 

### Syncing FineTune Jobs to W&B
 
 Logging Fine Tune with W&B to use later
 

In [ ]:
!openai wandb sync
wandb.finish()

wandb: Currently logged in as: gjain5 (cdi). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.10
wandb: Run data is saved locally in /content/wandb/run-20230307_195850-ft-Jim3RgfBVLSs8YTGfCwHxkFU
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-Jim3RgfBVLSs8YTGfCwHxkFU
wandb: ⭐️ View project at https://wandb.ai/cdi/GPT-3
wandb: 🚀 View run at https://wandb.ai/cdi/GPT-3/runs/ft-Jim3RgfBVLSs8YTGfCwHxkFU
wandb: Waiting for W&B process to finish... (success).
wandb: 
wandb: Run history:
wandb:             elapsed_examples ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:               elapsed_tokens ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:                training_loss ▅█▆▅▅▃█▇▅▃▄▂▇▄▄▃▆▆▂▅▄▃▄▂▄▂▅▁▄▂▄▂▂▁▂▄▅▃▄▃
wandb:   training_sequence_accuracy ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:      training_token_accuracy ▁▃▂▄▄▄▄▃▅▄▃▆▃▆▄▆▄▅▅▄▅▆▅▇▅▆▆█▃▆▆▆▇▆▅▇▄▇█▇
wandb:              validation_loss █▅▅▆▅█▆▆▅▄▃▇▇▄▄▅▄▁▅▇▆▅▅▅▅▃▅▇▂▅
